In [1]:
# Lớp chứa các hằng số đại diện cho các hệ điều hành
class Platform:
    WINDOWS = "windows"
    MACOS = "macos"

In [2]:
# platform = Platform.WINDOWS
platform = Platform.MACOS

In [3]:
# !pip install selenium

In [4]:
song_url = "https://www.hooktheory.com/theorytab/view/jay-hardway/stardust"
root_directory = "D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming"
root_directory = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming"

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import os

from selenium import webdriver
from bs4 import BeautifulSoup

import pandas as pd

import subprocess

import re

## Microsoft Edge Remote Debugging Port (tránh bị nhận diện phần mềm automation)

In [ ]:
macos_browser_data_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/browser_instance_data"
windows_browser_data_dir = "D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/browser_instance_data"

browser_instance_data_dir = macos_browser_data_dir if platform == Platform.MACOS else windows_browser_data_dir

port_to_run_browser = 40000

def open_browser_in_remote_debugging_mode(port, data_dir, platform=Platform.WINDOWS):
    macos_command = f'''open -na "Microsoft Edge.app" --args
                        --remote-debugging-port={port}
                        --user-data-dir="{data_dir}"'''

    windows_command = f'''start msedge.exe 
                        --remote-debugging-port={port} 
                        --user-data-dir="{data_dir}'''

    command = macos_command if platform == Platform.MACOS else windows_command
    
    os.system(command.replace("\n", " "))

open_browser_in_remote_debugging_mode(port_to_run_browser, browser_instance_data_dir, platform)

## Relative to Absolute path

In [6]:
def relative_to_absolute_path(relative_path, root_path=root_directory):
    absolute_path = f"{root_path}/{relative_path}"
    return absolute_path

## Kết nối Selenium đến trình duyệt Edge ở port được mở

In [ ]:
options = webdriver.EdgeOptions()
options.add_experimental_option("debuggerAddress", f"localhost:{port_to_run_browser}")
# options.add_argument("headless") # Chạy Selenium ở chế độ headless (không hiển thị giao diện browser)

browser = webdriver.Edge(options=options)
browser.maximize_window() # Phóng to cửa sổ trình duyệt

## Scroll xuống cuối trang với nhiều lần scroll cách nhau n giây

In [ ]:
def scroll_to_bottom(webdriver_instance=browser, scroll_pause_time = 0.5):
    # Get the screen height of the web
    screen_height = browser.execute_script("return window.screen.height;") 
    i = 1

    while True:
        # Scroll one screen height each time
        browser.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        time.sleep(scroll_pause_time)
        
        # Cập nhật scroll_height sau mỗi lần scroll, vì scroll_height có thể thay đổi sau khi scroll trang
        scroll_height = browser.execute_script("return document.body.scrollHeight;")

        i += 1
        if (screen_height) * i > scroll_height:
            break

## Lấy page source

In [ ]:
def get_html_content(url, webdriver_instance = browser):
    webdriver_instance.get(url)

    scroll_to_bottom(webdriver_instance)
    
    contents = webdriver_instance.page_source

    return contents

## Lấy page source tại đoạn nhạc được chỉ định

In [ ]:
def get_specific_part_of_a_song(html_content, part_of_the_song):
    """
    Lấy thông tin đoạn được chỉ định của bài hát cho trước
    :param html_content: nội dung html của trang web chứa bài hát
    :param part_of_the_song: đoạn cần lấy. Ví dụ: "Intro and Verse", "Chorus", v.v.
    :return: thẻ svg chứa phần của bài hát cần tìm
    """

    soup = BeautifulSoup(html_content, 'html.parser')

    h2 = soup.find('h2', string=part_of_the_song)

    svg = None

    if h2 and h2.parent.parent.parent.name == 'div':
        div = h2.parent.parent.parent.find_next_sibling("div")
        svg = div.find('svg', {'data-type': 'score'})
    else:
        pass
    
    return svg

In [ ]:
html_content = get_html_content(song_url, browser)
print(html_content)

# Save to file
def save_to_file(html_content, file_path):
    with open(file_path, "w") as f:
        f.write(html_content)

save_to_file(
    html_content, 
    relative_to_absolute_path("DataCrawling/html/Stardust.html")
)

In [ ]:
# Chạy thử get_next_div với url của bài hát
chorus_svg = get_specific_part_of_a_song(
    get_html_content(song_url, browser), 
    "Chorus"
)

# Kiểm tra kết quả
print(chorus_svg.prettify())

# Lưu nội dung html vào file
save_to_file(
    chorus_svg.prettify(), 
    relative_to_absolute_path("DataCrawling/html/Stardust - Chorus.html")
)


In [ ]:
# Lấy thẻ g có data-type đúng bằng note-staff
note_staff_g = chorus_svg.find('g', {'data-type': 'note-staff '})

# Xoá các thẻ g trong note_staff_g
for g in note_staff_g.find_all('g'):
    g.decompose()

# Prettyfy thẻ g in ra màn hình và lưu vào file
save_to_file(
    note_staff_g.prettify(), 
    relative_to_absolute_path("DataCrawling/html/Chorus - note-staff.html")
)

# Lấy tất cả thẻ rect trừ thẻ rect đầu tiên
rects = note_staff_g.find_all('rect')[1:]

# Đưa thông tin các thẻ rect vào dataframe với các cột lần lượt là các thuộc tính x, y, width, height, opacity
df = pd.DataFrame(
    data=[[rect["x"], rect["y"], rect["width"], rect["height"], rect["opacity"]] for rect in rects],
    columns=["x", "y", "width", "height", "opacity"]
)

# In dataframe
df

In [7]:
def html_file_to_beautiful_soup(file_path):
    with open(file_path, "r") as f:
        html_content = f.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        return soup
    
html_content = html_file_to_beautiful_soup(relative_to_absolute_path("DataCrawling/html/Chorus - note-staff.html"))
print(html_content.prettify())

<g data-type="note-staff ">
 <rect fill="rgb(255 255 255)" height="108" opacity="1" width="353.5" x="35" y="40">
  <title>
   Add a note or drag select
  </title>
 </rect>
 <rect fill="#4800FF" height="10" opacity="1" stroke="000000" stroke-width="0" style="pointer-events: none;" width="9.046875" x="36" y="133">
  <title>
  </title>
 </rect>
 <rect fill="#B800E5" height="10" opacity="1" stroke="000000" stroke-width="0" style="pointer-events: none;" width="9.046875" x="47.046875" y="122">
  <title>
  </title>
 </rect>
 <rect fill="#B800E5" height="10" opacity="1" stroke="000000" stroke-width="0" style="pointer-events: none;" width="9.046875" x="58.09375" y="122">
  <title>
  </title>
 </rect>
 <rect fill="#aaa" height="10" opacity="0" stroke="000000" stroke-width="0" style="pointer-events: none;" width="9.046875" x="69.140625" y="122">
  <title>
  </title>
 </rect>
 <rect fill="#ff0000" height="10" opacity="1" stroke="000000" stroke-width="0" style="pointer-events: none;" width="9.04687

In [ ]:
browser.quit()

In [ ]:
def windows_get_PID_of_process_running_on_port(port):
    command = f"netstat -a -n -o | findstr :{port}"

    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    #   TCP    127.0.0.1:40000        0.0.0.0:0              LISTENING       6920
    regex_pattern = r"\s+TCP\s+127.0.0.1:\d+\s+\d+.\d+.\d+.\d+:\d+\s+LISTENING\s+(.+)"

    match = re.search(regex_pattern, result.stdout)

    pid = None

    if match:
        pid = match.group(1).replace(" ", "")
    else:
        pid = None

    return pid

print(windows_get_PID_of_process_running_on_port(port_to_run_browser))

In [ ]:
def kill_process_running_on_port(port, platform=Platform.WINDOWS):
    result = None

    if platform == Platform.MACOS:
        result = os.system(f"lsof -ti tcp:{port} | xargs kill -9")
    else:
        pid = windows_get_PID_of_process_running_on_port(port)

        if pid:
            result = subprocess.run(
                f"taskkill /PID {pid} /F", 
                shell=True, 
                capture_output=True, 
                text=True
            ).stdout
        else:
            result = f"No process is running on port {port}"

    return result

print(kill_process_running_on_port(port_to_run_browser, Platform.WINDOWS))